In [1]:
from __future__ import print_function
import os
# Set the DISPLAY environment variable
os.environ["DISPLAY"] = ":0"
import e3sm_nex
import cdms2
import vcs
import jupyter
import requests
import numpy as np
from subprocess import Popen, PIPE
import shlex
import xarray as xr
import IPython.display

In [2]:
data_path = "/pscratch/sd/p/plutzner/E3SM/bigdata/v2.LR.historical_0101.eam.h0.1850-01.nc"
grid_file_path = "/pscratch/sd/p/plutzner/E3SM/bigdata/E3SMv2data/ne30pg2.nc"

In [3]:
# Reads in data from grid file
gf = cdms2.open(grid_file_path)

# needs to send filled data
lats = gf("grid_center_lat").filled()
lons = gf("grid_center_lon").filled()

lat_corners = gf("grid_corner_lat").filled()
lon_corners = gf("grid_corner_lon").filled()

In [4]:
# Generate grid
grid = e3sm_nex.generateMPASGrid(lats, lons, lat_corners, lon_corners)

In [5]:
# Load data and apply grid
fd = cdms2.open(data_path)
data = fd("TS")

# Need to clean up missing/bad values
data = cdms2.MV2.masked_greater(np.abs(data),1.e20)
data = e3sm_nex.applyGrid(data,grid)

In [6]:
# plot
# create Canvas
x=vcs.init(bg=True, geometry=(1200,800))
#x.backend.renWin.SetOffScreenRendering(True)

# Create graphics method
mesh = vcs.createmeshfill() # THIS LINE CRASHES THE KERNEL

# Only show a sub domain
mesh.datawc_x1 = -170
mesh.datawc_x2 = -50
mesh.datawc_y1 = -25
mesh.datawc_y2 = 75

# If you want to show mesh
mesh.mesh = True

x.plot(data, mesh, ratio="autot")

# Save the plot to a file
x.png('plot.png')

# Display the saved image in Jupyter notebook
IPython.display.Image('plot.png')